In [1]:
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.models import Sequential
from keras.layers import LSTM,Dense,Input,ConvLSTM2D,Reshape,Activation,Lambda,Softmax,Concatenate
from keras.layers import Dense, LSTM, Input, Embedding, TimeDistributed, Flatten, Dropout,RepeatVector,Reshape,Activation,Lambda,Softmax
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from sklearn.model_selection import train_test_split
import re
import pandas as pd
from keras import backend as K
import keras.backend as kerback
#from keras.models import Model
from tensorflow.keras.models import Model
from keras.preprocessing.text import one_hot
import tensorflow 

In [2]:
# train a generative adversarial network on a one-dimensional function
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy import random
from numpy.random import rand
from numpy.random import randn
from keras.models import Sequential
from keras.layers import Dense

In [3]:
def prepare_data(f):
    data = pd.read_csv(f, sep="\n",header=None)
    list_listword=[]#liste des listes des mots de chaque tweet
    list_tweets=data[0].values.tolist()#liste des tweets
    l=[]
    for text in list_tweets:
        text = re.sub(r'http\S+', '', text)   # Remove URLs
        text = re.sub(r'—', ' ', text) 
        text = re.sub(r'@[a-zA-Z0-9_]+', '', text)  # Remove @ mentions
        text = text.strip(" ")   # Remove whitespace resulting from above
        text = re.sub(r' +', ' ', text)   # Remove redundant spaces
        l.append(text)
    
    return l

In [4]:
list_tweets=prepare_data('C:/Users/LENOVO/Desktop/PCD/tweet.txt')

In [5]:
def tokenizer_data(list_tweets):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(list_tweets)
    sequences = tokenizer.texts_to_sequences(list_tweets)
    vocab_size=len(tokenizer.word_index)
    return sequences, tokenizer, vocab_size

def max_tweet(sequences):
    max_length=0
    for i in range(len(sequences)):
        if(len(sequences[i])>max_length):
            max_length=len(sequences[i])
    return max_length

sequences, tokenizer, vocab_size=tokenizer_data(list_tweets)
max_length=max_tweet(sequences)
sequences=np.array([xi+[0]*(max_length-len(xi)) for xi in sequences])
y=sequences[:,-1]
print(y.shape)
y_train = y.reshape(-1, 1)
X=sequences
X_train = np.expand_dims(X, axis=2)
nb_tweet=len(sequences)
batch_size=max_length
print(vocab_size)

(66,)
841


In [6]:
 import keras.backend as kerback
# define the standalone generator model
def build_generator(n_outputs):#max_length nombre d'element das une tweet
   
    noise=Input(shape=(57,1))
    
    label=Input(shape=(57,))
    label_emb=Embedding(vocab_size,1)(label)
    
    x0=Concatenate(axis=2)([noise,label_emb])
    x1=LSTM(15, activation='relu',input_dim=n_outputs)(x0)
    #15 nombre de neurone ,batch_size=n_batch=20 
    # repeat vector
    x2=RepeatVector(n_outputs)(x1)
    # decoder layer
    x3=LSTM(15, activation='relu', return_sequences=True)(x2)
    x4=TimeDistributed(Dense(1))(x3)
    unstacked = Lambda(lambda x: tensorflow.unstack(x, axis=2))(x4)
    dense_outputs = [Dense(n_outputs)(x) for x in unstacked]
    merged = Lambda(lambda x: K.stack(x, axis=2))(dense_outputs)

    #generated_tweet=model(x0)
    model = Model([noise,label],merged)
    #model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    print('generator')
    print(model.summary())
    return  model

build_generator(57)

generator
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 57)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 57, 1)]      0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 57, 1)        841         input_2[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 57, 2)        0           input_1[0][0]                    
                                                                 embedding[0]

In [7]:
# define the standalone discriminator model
def build_discriminator(n_inputs):
    tweet=Input(shape=(n_inputs,1)) #n_imput nombre des element 
    
    label=Input(shape=(n_inputs,))
    label_emb=Embedding(vocab_size,1)(label)
    
    x0=Concatenate(axis=1)([tweet,label_emb])
    x2=LSTM(15, activation='relu', input_dim=n_inputs)(x0)
    x3=RepeatVector(n_inputs)(x2)
    x4=LSTM(15, activation='relu', return_sequences=True)(x3)
    x5=TimeDistributed(Dense(1))(x4)
    unstacked = Lambda(lambda x: tensorflow.unstack(x, axis=2))(x5)
    dense_outputs = [Dense(n_inputs)(x) for x in unstacked]
    merged = Lambda(lambda x: K.stack(x, axis=2))(dense_outputs)
    x1=Flatten()(merged)
    x6=Dense(1, activation='sigmoid')(x1)
    model = Model([tweet,label], x6)
    print(model.summary())
    model.compile(optimizer='adam', loss='binary_crossentropy' ,metrics=['accuracy'])
    return model
build_discriminator(57)

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 57)]         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 57, 1)]      0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 57, 1)        841         input_4[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 114, 1)       0           input_3[0][0]                    
                                                                 embedding_1[0][0]     

In [8]:
# define the combined generator and discriminator model
def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    # connect them
    model = Sequential()
    # add generator
    #model.add(generator)
    x0=Input(shape=(57,1))
    x1=Input(shape=(57,))
    x2=generator([x0,x1])
    print(x2[-1].shape)
    x3=discriminator([x2,x1])
    print(x3.shape)
    model = Model([x0,x1], x3)
    # add the discriminator
    #model.add(discriminator)
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    print(model.summary())
    return model
define_gan(build_generator(57), build_discriminator(57))

generator
Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 57)]         0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 57, 1)]      0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 57, 1)        841         input_6[0][0]                    
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 57, 2)        0           input_5[0][0]                    
                                                                 embedding_2[

In [9]:
def generate_real_samples():
    list_tweets=prepare_data('C:/Users/LENOVO/Desktop/PCD/tweet.txt')
    sequences, tokenizer, vocab_size=tokenizer_data(list_tweets)
    max_length=max_tweet(sequences)
    sequences=np.array([xi+[0]*(max_length-len(xi)) for xi in sequences])
    y=sequences[:,-1]
    #y_train = y.reshape(-1, 1)
    X=sequences
    X_train = np.expand_dims(X, axis=2)
    #print(X_train.shape)
    nb_tweet=len(sequences)
    batch_size=max_length
    
    return [X_train,y_train]
generate_real_samples()

[array([[[146],
         [  7],
         [ 46],
         ...,
         [  0],
         [  0],
         [  0]],
 
        [[ 87],
         [ 12],
         [  4],
         ...,
         [  0],
         [  0],
         [  0]],
 
        [[ 91],
         [ 59],
         [ 12],
         ...,
         [  0],
         [  0],
         [  0]],
 
        ...,
 
        [[ 68],
         [  3],
         [320],
         ...,
         [  0],
         [  0],
         [  0]],
 
        [[ 25],
         [  8],
         [ 24],
         ...,
         [  0],
         [  0],
         [  0]],
 
        [[834],
         [305],
         [  1],
         ...,
         [  6],
         [ 61],
         [ 62]]]),
 array([[ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],

In [10]:
# Create batch labels to use when calling train_on_batch
valid = ones((max_length,1))
fake = zeros((max_length,1))

In [11]:
generator = build_generator(max_length)
# Build and compile the discriminator
discriminator = build_discriminator(max_length)
#discriminator.compile(optimizer='adam', loss='binary_crossentropy' ,metrics=['accuracy'])

# Build and compile the combined model
gan_model = define_gan(generator, discriminator)

generator
Model: "functional_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 57)]         0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, 57, 1)]      0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 57, 1)        841         input_12[0][0]                   
__________________________________________________________________________________________________
concatenate_4 (Concatenate)     (None, 57, 2)        0           input_11[0][0]                   
                                                                 embedding_4

In [12]:
def sample_tweet(epoch,noise,sampled_labels,g_model):
    result = g_model.predict([noise,sampled_labels])
    #print(result)
    #print(result.shape)
    #print(len(result))
    res=[]
    predicted_word =""
    for j in range(57):
        #predicted_word = ''
        for word, index in tokenizer.word_index.items():
            if index == int(result[0][j]):
                predicted_word = predicted_word+' '+word
                break
    res.append(predicted_word)
    print(res)
    
    #for i in res:
    #    print(i)
    return res

In [13]:
def disc_text(model_disc, seed_text):
    #print(seed_text)
    sequences, tokenizer=tokenizer_data(seed_text)
    sequences= pad_sequences(sequences, maxlen=max_length, padding='post')
    y = model_disc.predict(sequences)
    #print(y)
    #for i in y:
    if(i<0.7):
        print("fake")
    else:
        print("real")

In [ ]:
# Store the losses
d_losses = []
g_losses = []
sample_period = 100 # every `sample_period` steps generate and save some data"
Epoch=3000
#x_real= generate_real_samples()
######################## Main training loop###########################
for i in range(Epoch):
        ########################
        ###Train Discriminator##
        #######################
        # prepare real samples
        [X_train,y_train]= generate_real_samples()
        #print(X_train.shape)
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        #print(idx)
        tweets, labels = X_train[idx], y_train[idx]
        #print(tweets.shape)
        #print(labels.shape)
        # Sample noise as generator input
        noise = np.random.normal(0, 1, (batch_size, 1))
        #print(noise.shape)
        # Generate fake tweets
        gen_twt= generator.predict([noise,labels])
        
        
        # Train the discriminator
        # both loss and accuracy are returned
        d_loss_real, d_acc_real=discriminator.train_on_batch([tweets,labels], valid)
        d_loss_fake, d_acc_fake =discriminator.train_on_batch([gen_twt,labels], fake)
        d_loss = 0.5 * (d_loss_real + d_loss_fake)
        d_acc  = 0.5 * (d_acc_real + d_acc_fake)
        
        #######################
        ### Train generator ###
        #######################
    
        # Condition on labels
        sampled_labels = np.random.randint(0, 10, batch_size)                          
        #print(sampled_labels.shape)                            
        y_gan = ones((57, 1))
        # update the generator via the discriminator's error
        noise=random.randn(57,1)
        #gan_model([noise,x_real[-1]], y_gan).summary()
        g_loss, n=gan_model.train_on_batch([noise,sampled_labels], y_gan)
        
        # Save the losses
        d_losses.append(d_loss)
        g_losses.append(g_loss)
        
        #if i % 2 == 100:#after 100 epoch we save
        #if i % sample_period == 0:
        print(f"epoch: {i+1}, d_loss: {d_loss:.2f}, \
        d_acc: {d_acc:.2f}, g_loss: {g_loss:.2f}")
        sample_tweet(i,noise,sampled_labels, generator)
        #disc_text(discriminator, tex)

epoch: 1, d_loss: 0.69,         d_acc: 0.21, g_loss: 0.69
['']
epoch: 2, d_loss: 0.66,         d_acc: 0.47, g_loss: 0.68
['']
epoch: 3, d_loss: 0.61,         d_acc: 0.50, g_loss: 0.68
['']
epoch: 4, d_loss: 0.59,         d_acc: 0.50, g_loss: 0.68
['']
epoch: 5, d_loss: 0.59,         d_acc: 0.48, g_loss: 0.69
['']
epoch: 6, d_loss: 0.55,         d_acc: 0.50, g_loss: 0.69
['']
epoch: 7, d_loss: 0.49,         d_acc: 0.50, g_loss: 0.69
['']
epoch: 8, d_loss: 0.49,         d_acc: 0.50, g_loss: 0.70
['']
epoch: 9, d_loss: 0.50,         d_acc: 1.00, g_loss: 0.70
['']
epoch: 10, d_loss: 0.45,         d_acc: 0.98, g_loss: 0.71
['']
epoch: 11, d_loss: 0.44,         d_acc: 0.98, g_loss: 0.71
['']
epoch: 12, d_loss: 0.45,         d_acc: 0.97, g_loss: 0.72
['']
epoch: 13, d_loss: 0.43,         d_acc: 0.97, g_loss: 0.73
['']
epoch: 14, d_loss: 0.42,         d_acc: 0.98, g_loss: 0.74
['']
epoch: 15, d_loss: 0.37,         d_acc: 0.98, g_loss: 0.75
['']
epoch: 16, d_loss: 0.39,         d_acc: 0.96, g_l

epoch: 78, d_loss: 0.06,         d_acc: 1.00, g_loss: 2.33
['']
epoch: 79, d_loss: 0.10,         d_acc: 0.97, g_loss: 2.36
['']
epoch: 80, d_loss: 0.09,         d_acc: 0.98, g_loss: 2.39
['']
epoch: 81, d_loss: 0.12,         d_acc: 0.97, g_loss: 2.41
['']
epoch: 82, d_loss: 0.05,         d_acc: 1.00, g_loss: 2.44
['']
epoch: 83, d_loss: 0.07,         d_acc: 0.99, g_loss: 2.48
['']
epoch: 84, d_loss: 0.11,         d_acc: 0.97, g_loss: 2.51
['']
epoch: 85, d_loss: 0.06,         d_acc: 0.99, g_loss: 2.54
['']
epoch: 86, d_loss: 0.08,         d_acc: 0.98, g_loss: 2.57
['']
epoch: 87, d_loss: 0.09,         d_acc: 0.98, g_loss: 2.60
['']
epoch: 88, d_loss: 510035.57,         d_acc: 0.57, g_loss: 2.80
['']
epoch: 89, d_loss: 0.05,         d_acc: 0.99, g_loss: 2.97
['']
epoch: 90, d_loss: 0.12,         d_acc: 0.96, g_loss: 3.10
['']
epoch: 91, d_loss: 0.16,         d_acc: 0.96, g_loss: 3.21
['']
epoch: 92, d_loss: 0.24,         d_acc: 0.91, g_loss: 3.27
['']
epoch: 93, d_loss: 0.41,         d_

epoch: 205, d_loss: 0.46,         d_acc: 0.81, g_loss: 1.15
['']
epoch: 206, d_loss: 0.45,         d_acc: 0.80, g_loss: 1.16
['']
epoch: 207, d_loss: 0.43,         d_acc: 0.82, g_loss: 1.16
['']
epoch: 208, d_loss: 0.44,         d_acc: 0.81, g_loss: 1.16
['']
epoch: 209, d_loss: 0.41,         d_acc: 0.82, g_loss: 1.17
['']
epoch: 210, d_loss: 0.50,         d_acc: 0.75, g_loss: 1.17
['']
epoch: 211, d_loss: 0.50,         d_acc: 0.73, g_loss: 1.17
['']
epoch: 212, d_loss: 0.40,         d_acc: 0.86, g_loss: 1.17
['']
epoch: 213, d_loss: 0.44,         d_acc: 0.79, g_loss: 1.17
['']
epoch: 214, d_loss: 0.43,         d_acc: 0.80, g_loss: 1.18
['']
epoch: 215, d_loss: 0.50,         d_acc: 0.75, g_loss: 1.17
['']
epoch: 216, d_loss: 0.45,         d_acc: 0.80, g_loss: 1.18
['']
epoch: 217, d_loss: 0.46,         d_acc: 0.78, g_loss: 1.17
['']
epoch: 218, d_loss: 0.51,         d_acc: 0.74, g_loss: 1.17
['']
epoch: 219, d_loss: 0.47,         d_acc: 0.81, g_loss: 1.17
['']
epoch: 220, d_loss: 0.44,

epoch: 296, d_loss: 2.78,         d_acc: 0.53, g_loss: 1.15
[' to to to to to to to to to to the to to the the the the the the the the the and the the and and and and and a a a a in a in in in a the and the the the to to']
epoch: 297, d_loss: 2.95,         d_acc: 0.45, g_loss: 2.53
[' to to to to the to the the the to to the and the the and and and and and and and a and a and and a a a in a of of of of our of of our our of the a and and and the to']
epoch: 298, d_loss: 3.13,         d_acc: 0.47, g_loss: 2.62
[' to to to to the to the the to to the and the the and and and and and and and and and a and and a a a a a of of of of of of of of our of the a and and and the to']
epoch: 299, d_loss: 3.03,         d_acc: 0.51, g_loss: 3.74
[' to to to the the the the the the to to the and the the and and and and a and a a a a a a a a a in in of of our our our our our our our of and in and and and the to']
epoch: 300, d_loss: 3.90,         d_acc: 0.34, g_loss: 2.09
[' to to to the the the the the

epoch: 333, d_loss: 0.36,         d_acc: 0.77, g_loss: 2.08
[' the the the and a to the and to to a in our of you our this this our you this it it this this it this it for for vote is vote vote vote this is is is it and you the the the']
epoch: 334, d_loss: 0.42,         d_acc: 0.68, g_loss: 2.05
[' to to to the the to to to to the and a a in a in in a a in in of in in of in of of of our our our our our in our our our of the in to to to']
epoch: 335, d_loss: 0.43,         d_acc: 0.68, g_loss: 1.91
[' to to to to the to to to the and a and in a in in a a in in of in in in in of of of our our our our our in of our our in the in to to to']
epoch: 336, d_loss: 0.38,         d_acc: 0.72, g_loss: 1.83
[' to to to the the to to the to the and in a in a of in a in in of of in of of of of our our our our you you you of our our our of the in to to to']
epoch: 337, d_loss: 0.40,         d_acc: 0.71, g_loss: 1.90
[' to to to to to to to the the a and a and in a and a a in in a in in a in in in of 

...

epoch: 461, d_loss: 0.47,         d_acc: 0.96, g_loss: 0.89
[' to to to to the and and a and a a and and a a a and a a and a a a a and and and and to to to to']
epoch: 462, d_loss: 0.53,         d_acc: 0.93, g_loss: 0.99
[' to to the the the and the and and the the and and and and and and the and and and and the the the the to to to to']
epoch: 463, d_loss: 0.46,         d_acc: 0.91, g_loss: 0.90
[' to to to to the and and a and a a and and a a a and a a and a a a a and and and and to to to to']
epoch: 464, d_loss: 0.47,         d_acc: 0.92, g_loss: 0.93
[' to to the the and the and and the the the and and the the and the the and the and the the the the to to to']
epoch: 465, d_loss: 0.52,         d_acc: 0.93, g_loss: 0.85
[' to to the and the and and and and the and and and and and and and and and and and and and and the and to to to to']
epoch: 466, d_loss: 0.50,         d_acc: 0.87, g_loss: 0.90
[' to to to to to to the and a a in a in in a a in in in a in in a in in in in a a a a t

epoch: 513, d_loss: 0.52,         d_acc: 0.76, g_loss: 0.88
[' to to to to to to to to to to']
epoch: 514, d_loss: 0.55,         d_acc: 0.73, g_loss: 0.84
['']
epoch: 515, d_loss: 0.53,         d_acc: 0.73, g_loss: 0.85
[' to to the the and the and and the the the and and the the and the the the the the the the the the to']
epoch: 516, d_loss: 0.51,         d_acc: 0.79, g_loss: 0.87
['']
epoch: 517, d_loss: 0.55,         d_acc: 0.72, g_loss: 0.84
['']
epoch: 518, d_loss: 0.55,         d_acc: 0.74, g_loss: 0.85
[' to to to to to']
epoch: 519, d_loss: 0.50,         d_acc: 0.76, g_loss: 0.84
[' to to to to to the to to to to to to to to to to to to to to to to to to']
epoch: 520, d_loss: 0.57,         d_acc: 0.70, g_loss: 0.85
[' to to to the to the the to to to the the to to to to to to to to to to to to']
epoch: 521, d_loss: 0.50,         d_acc: 0.80, g_loss: 0.84
[' to to to to to to to to to to to to to to to to to to to to to to to']
epoch: 522, d_loss: 0.54,         d_acc: 0.72, g_l

epoch: 605, d_loss: 0.47,         d_acc: 0.81, g_loss: 0.95
['']
epoch: 606, d_loss: 0.52,         d_acc: 0.73, g_loss: 0.95
['']
epoch: 607, d_loss: 0.57,         d_acc: 0.66, g_loss: 0.94
[' to to to the to the the to to to the the to to to to to to to to to to to to']
epoch: 608, d_loss: 0.54,         d_acc: 0.73, g_loss: 0.95
['']
epoch: 609, d_loss: 0.53,         d_acc: 0.70, g_loss: 0.94
['']
epoch: 610, d_loss: 0.51,         d_acc: 0.75, g_loss: 0.94
['']
epoch: 611, d_loss: 0.49,         d_acc: 0.78, g_loss: 0.94
[' to to to to to to to to to to to to to to to to to to to to to to']
epoch: 612, d_loss: 0.50,         d_acc: 0.74, g_loss: 0.94
[' to to to to to to to to to to to to to to to to to to to to to to to']
epoch: 613, d_loss: 0.52,         d_acc: 0.73, g_loss: 0.94
['']
epoch: 614, d_loss: 0.60,         d_acc: 0.67, g_loss: 0.94
['']
epoch: 615, d_loss: 0.53,         d_acc: 0.72, g_loss: 0.94
[' to to to to to to to to to to to to to to to to to to to to to']
epoch: 616

epoch: 690, d_loss: 0.49,         d_acc: 0.76, g_loss: 1.00
[' to to to the to the the to to the the the to the the to the the to the to to to to']
epoch: 691, d_loss: 0.49,         d_acc: 0.75, g_loss: 1.01
[' to to to to to to to to to to to to to to to to to to to']
epoch: 692, d_loss: 0.44,         d_acc: 0.80, g_loss: 1.01
['']
epoch: 693, d_loss: 0.49,         d_acc: 0.78, g_loss: 1.01
[' to to to to to to to to to to to to to to to to to to to to to to to']
epoch: 694, d_loss: 0.45,         d_acc: 0.81, g_loss: 1.01
['']
epoch: 695, d_loss: 0.52,         d_acc: 0.71, g_loss: 1.00
[' to to to to to to to to to to to to to to to to to to to to to to']
epoch: 696, d_loss: 0.49,         d_acc: 0.75, g_loss: 1.02
['']
epoch: 697, d_loss: 0.51,         d_acc: 0.73, g_loss: 1.01
[' to to to the the and the and and the the and and and the and and the and and the and the the to the']
epoch: 698, d_loss: 0.46,         d_acc: 0.78, g_loss: 1.01
['']
epoch: 699, d_loss: 0.56,         d_acc:

epoch: 772, d_loss: 0.50,         d_acc: 0.75, g_loss: 1.07
[' to to the the the the and the the the the the and the the the the the the the the the the to the']
epoch: 773, d_loss: 0.47,         d_acc: 0.79, g_loss: 1.06
[' to to to to']
epoch: 774, d_loss: 0.51,         d_acc: 0.73, g_loss: 1.06
[' to to to to to to to to to to to to to to to to to to to']
epoch: 775, d_loss: 0.43,         d_acc: 0.82, g_loss: 1.06
[' to to to to to the to to to to the the to to to to to to to to to to to to']
epoch: 776, d_loss: 0.47,         d_acc: 0.78, g_loss: 1.05
[' to to the the the the and the the the the the the the the the the the the the the the the to to']
epoch: 777, d_loss: 0.47,         d_acc: 0.78, g_loss: 1.06
[' to to to to']
epoch: 778, d_loss: 0.45,         d_acc: 0.82, g_loss: 1.06
[' to to to to to to to to to to to to to to to to to to to to to to to']
epoch: 779, d_loss: 0.48,         d_acc: 0.75, g_loss: 1.06
[' to to the the and the and and the the the and and the the the th